In [1]:
# First, install autokeras
!pip install autokeras --quiet

In [2]:
!pip install gradio --quiet

In [3]:
# need for a few specific things
import numpy # We need this to massage some inputs
import tensorflow.keras.models # We need this to load a model after it is saved

# Core to what we're doing
import sklearn.datasets # We need this to load a sample dataset
import autokeras # We need this to do some ML
import gradio # We need this to build a GUI

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [5]:
import pandas

dataset = pandas.read_csv("/content/drive/MyDrive/colab_notebook/penguins_size.csv")
dataset.head()

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


In [6]:
species = ['Adelie', 'Chinstrap', 'Gentoo']

In [7]:
labels = dataset['species']
features = dataset.iloc[:,1:]

In [8]:
# Initialize the structured data classifier.
clf = autokeras.StructuredDataClassifier(
    max_trials=10, # We will do 10 iterations of model improvement
    overwrite=True, # Overwrite prior training attempts
) 

# Fit the model to the data that we used earlier
clf.fit(
    x = features, # Measurements of iris dimensions
    y = labels, # Iris species
    epochs=5 # The number of iterations to train each attempted model
)

Trial 10 Complete [00h 00m 03s]
val_accuracy: 0.0

Best val_accuracy So Far: 0.9821428656578064
Total elapsed time: 00h 00m 34s
Epoch 1/5
11/11 [==============================] - 1s 3ms/step - loss: 1379.6078 - accuracy: 0.1977
Epoch 2/5
11/11 [==============================] - 0s 3ms/step - loss: 850.9223 - accuracy: 0.1977
Epoch 3/5
11/11 [==============================] - 0s 3ms/step - loss: 501.4278 - accuracy: 0.1163
Epoch 4/5
11/11 [==============================] - 0s 3ms/step - loss: 388.9028 - accuracy: 0.4390
Epoch 5/5
11/11 [==============================] - 0s 3ms/step - loss: 271.4534 - accuracy: 0.4390


In [10]:
model = clf.export_model()
model.save("model_autokeras.tf")

In [11]:
# Load the model
loaded_model = tensorflow.keras.models.load_model("model_autokeras.tf", custom_objects=autokeras.CUSTOM_OBJECTS)

In [12]:
def predict(island, culmen_length_mm, culmen_depth_mm,	flipper_length_mm,	body_mass_g,	sex):
  # Format a model input 
  input = numpy.array([[island, culmen_length_mm, culmen_depth_mm,	flipper_length_mm,	body_mass_g,	sex]])
  # Make a prediction using the model. This returns a numeric integer output (e.g., 1)
  pred = loaded_model.predict(input)
  # Transform that into a species prediction
  species_pred = species[int(pred[0][0])]
  # Return the answer
  return species_pred

In [13]:
predict('Torgersen', 39.1, 18.7, 181.0, 3750.0, 'MALE')

'Adelie'

In [ ]:
demo = gradio.Interface(
    fn = predict, # Define the function connecting inputs to outputs
    inputs = ["text", "number", "number", "number", "number", "text"], # Define the type and number of inputs you'd like to have
    outputs = gradio.Textbox(label = "species"), # Define the number and type of outputs you'd like to have
    examples = [['Torgersen',	39.1,	18.7,	181.0, 3750.0, 'MALE']]
)
 
# Launch the interface
demo.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://15008.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces
